In [ ]:
import os
import numpy as np
import pickle as pcl
import matplotlib
#matplotlib.use('agg')
from matplotlib import pyplot as plt
from IPython.display import clear_output
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

from predict import predict

In [ ]:
si = predict(fname='data/full_histories_illustris.h5')
si.training_mask()

In [ ]:
sn = 50
predictors = si.load_arr('SFH/log_8')
illustris_dust, wl = si.load_spectra('Dust')
illustris_noise = np.random.normal(loc=0, scale=illustris_dust / sn)
illustris_final_dust_noise = illustris_dust + illustris_noise
si.generate_standardisation(key='Dust Noise SN50', spec=illustris_final_dust_noise)
features = si.prepare_features(illustris_final_dust_noise, key='Dust Noise SN50', RNN=True)
model,scores = si.create_rnn_model(features, predictors, batch_size=20, train=si.train)

In [ ]:
f = 'data/rnn_trained_illustris_dust_noise50.h5'
if os.path.isfile(f): os.remove(f)
model.save(f)